# Rakuten USA - Data Engineering Test

### Goal
The goal of this test is not ment to be exhaustive, but we do want to test your skills in analytical thinking, SQL, data wrangling, data analysis, and data visualization.

### Expectations
You can expect this test to use the following technologies:
- Python3
- Jupyter Notebook
- PySpark
- PySpark SQL

This test will cover the following areas: 
- Data cleaning/wrangling
- Exploratory Data Analysis (EDA)
- Data Visualization (Bonus)
- Lessons learned

This test should take you no more than 2-4 hours.

### What do you need to turn in, and what should I turn in?
Please email stirling.waite@rakuten with your finished notebook
- rakuten_engineering_test.ipynb
 
### Questions?
Feel free to email me or call me with any questions about this test
- stirling.waite@rakuten.com
- 801-856-1426

# About The Dataset

Below you will find a free Kaggle competition dataset from https://www.kaggle.com/mayank811/employee-reviews/data using scraped data from Glassdoor.com.

## Context
- Over 67k employee reviews for Google, Amazon, Facebook, Apple, Microsoft, and Netflix.
- Web scraped from Glassdoor

## Content
This dataset contains employee reviews separated into the following categories:

- **Index:** index
- **Company:** Company name
- **Location:** This dataset is global, as such it may include the country's name in parenthesis [i.e "Toronto, ON(Canada)"]. However, if the location is in the USA then it will only include the city and state[i.e "Los Angeles, CA" ]
- **Date Posted:** in the following format MM DD, YYYY
- **Job-Title:** This string will also include whether the reviewer is a 'Current' or 'Former' Employee at the time of the review
- **Summary:** Short summary of employee review
- **Pros:** Pros
- **Cons:** Cons
- **Overall Rating:** 1-5
- **Work/Life Balance Rating:** 1-5
- **Culture and Values Rating:** 1-5
- **Career Opportunities Rating:** 1-5
- **Comp & Benefits Rating:** 1-5
- **Senior Management Rating:** 1-5
- **Helpful Review Count:** 1-5

**Link to Review:** This will provide you with a direct link to the page that contains the review. However it is likely that this link will be outdated

**NOTE:** 'none' is placed in all cells where no data value was found.



# PySpark Initialization

In [1]:
import re
import pyspark
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import *
from pyspark.sql import functions as fn
from pyspark.sql import types as t
from pyspark.sql.types import *
from datetime import datetime

In [2]:
conf = SparkConf()\
                .setMaster("local[*]")\
                .setAppName("rakutenHomework")
        
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
sqlContext = SQLContext(sc)
spark = SparkSession.builder.appName("spark play").getOrCreate()

## Read Dataset into Spark

In [40]:
reviews_df = spark.read.csv("employee_reviews_usa.csv", header=True, mode="DROPMALFORMED", inferSchema='true', encoding="utf-8")
print(reviews_df.count())
reviews_df.show()

27779
+---+-------+-----------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+------------------+--------------------+--------------------------+------------------+----------------------+-------------+--------------------+
|_c0|company|         location|        dates|           job_title|             summary|                pros|                cons|      advice_to_mgmt|overall_ratings|work_balance_stars|culture_values_stars|carrer_opportunities_stars|comp_benefit_stars|senior_mangemnet_stars|helpful_count|                link|
+---+-------+-----------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+------------------+--------------------+--------------------------+------------------+----------------------+-------------+--------------------+
|  0| google|Mountain View, CA|  Feb 8, 2015|Current Employee

# Data Cleaning & Removing Un-Needed Columns

No one likes dirty data.  Let's clean up this dataset to make it more usable for our use case.

 
1. Make the date field more data friendly
    - Hint #1: Use the provided `review_date_udf` udf provided (it will make your life more simple) 
2. Using the "location" column 
    - Add new columns for city, state, country columns
        - Hint #1: If location is not in US it will show as Toronto, ON(Canada)
        - Hint #2: Use something like the following to input the `country` column
            - ```lit("US").alias("country")```
3. Using the "job-title" column  
    - Add a new boolean column titled "current_employee" that will show:
        - 1,  if reviewer is a current employee
        - 0,  if reviewer is not a current employee
    - Remove "Former Employee - " and "" from the "job-title" column
        - Hint: Use the udf function `reg_job_title_udf`
4. Drop un-needed columns 
    - "_c0", "link", "pros", "cons", "advice-to-mgmt", "summary", "location"

##### Useful UDF functions needed for below

In [43]:
review_date_udf = fn.udf(
    lambda x: datetime.strptime(x, ' %b %d, %Y'), DateType()
)

In [44]:
def reg_job_title(column):
    res_split=[]
    reg_patterns=["Former Employee - / ", "Current Employee - / "]
    for i in range(len(reg_patterns)):
        res_split=re.findall(r"[^/]+",reg_patterns[i])
        for x in res_split[0].split("|"):
            column = column.replace(x,res_split[1])
    return column[1:]

reg_job_title_udf = fn.udf(reg_job_title, t.StringType())

In [54]:
cleaned_reviews_df = reviews_df.where((col("dates") != 'None') & (col("dates") != ' Jan 0, 0000') & (col("dates") != ' Nov 0, 0000') & (~col("location").contains("(")) & (col("location") != "none")) \
                         .select(
                             col("company")
                         )

cleaned_reviews_df.show()

+-------+-------------+-----+-------+-----------+--------------------+----------------+---------------+------------------+--------------------+--------------------------+------------------+----------------------+-------------+
|company|         city|state|country|review_date|           job_title|current_employee|overall_ratings|work_balance_stars|culture_values_stars|carrer_opportunities_stars|comp_benefit_stars|senior_mangemnet_stars|helpful_count|
+-------+-------------+-----+-------+-----------+--------------------+----------------+---------------+------------------+--------------------+--------------------------+------------------+----------------------+-------------+
| google|Mountain View|   CA|     US| 2015-02-08|  Anonymous Employee|               1|            5.0|               2.0|                 5.0|                       5.0|               4.0|                   5.0|          498|
| google|Mountain View|   CA|     US| 2018-12-09|                SDE2|               0|     

# Testing Questions

Each question below will have two parts.   Please solve each problem in PySpark and PySpark SQL.  This is to test your ability to solve the problem in PySpark, and to test your ability to solve the problem in SQL.

- PySpark API: https://spark.apache.org/docs/latest/api/python/index.html
- PySpark SQL API: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html

In [61]:
cleaned_reviews_df.createOrReplaceTempView("reviews")

### (1) What is the total number of reviews in the data set?

##### PySpark Implmentation

In [47]:
cleaned_reviews_df.count()

27779

##### Spark SQL Implmentation

In [48]:
spark.sql("SELECT count(*) as review_count from reviews").show()

+------------+
|review_count|
+------------+
|       27779|
+------------+



###  (2) How many reviews exist in the dataset per each company?

##### PySpark Implmentation

##### Spark SQL Implmentation

###  (3) How many reviews are from former and current employees per each company?

##### PySpark Implmentation

##### Spark SQL Implmentation

###  (4) What is the average review scores of each current and former employees for each company?

##### PySpark Implmentation

##### Spark SQL Implmentation

###  (5) Show the following in one solution:
1. Get all the former Google employee reviewers in locations Seattle, San Francisco, or Utah

- Also, get the count and average work_balance_score for reviews in #1

##### PySpark Implmentation

##### Spark SQL Implmentation

# Visualize Your Data (Bonus Points)
1. Turn your Spark dataframe into a Pandas dataframe
2. Answer the visualization questions with any of the following libraries:
    - Matplotlib
    - Seaborn
    - Bookeh

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

In [21]:
reviews_pd_df = cleaned_reviews_df.toPandas()
print(reviews_pd_df)

         company           city state country review_date  \
0         google  Mountain View    CA      US  2015-02-08   
1         google  Mountain View    CA      US  2018-12-09   
2         google       New York    NY      US  2018-12-11   
3         google       New York    NY      US  2018-12-10   
4         google  Mountain View    CA      US  2018-12-08   
5         google  San Francisco    CA      US  2018-12-03   
6         google       New York    NY      US  2018-11-26   
7         google    Los Angeles    CA      US  2018-11-29   
8         google  San Francisco    CA      US  2018-11-17   
9         google  Mountain View    CA      US  2018-12-09   
10        google  Mountain View    CA      US  2018-12-08   
11        google       Dearborn    MI      US  2018-11-20   
12        google       New York    NY      US  2018-12-06   
13        google      Sunnyvale    CA      US  2018-11-21   
14        google        Seattle    WA      US  2018-11-13   
15        google       N

### Plot the distribution of reviewers who are current and former employees
- Hint: A Seaborn factorplot will work nicely here

### Plot the distributions of reviews for each company (current and former employees) and their overall ratings
- Hint: A Seaborn factorplot will work nicely here

# Lessons Learned
- Enter anything below you want us to know about the test